In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

plt.rcParams['font.size'] = 16

In [ ]:
exp_order = pd.read_csv("exp_order.csv").values
exp_answer = pd.read_csv("exp_answer.csv").values[:,5:17].astype(float)
pra = pd.read_csv("exp_answer.csv").values[:,1:5].astype(float)
exp_answer.shape

In [ ]:
# Sort
sorted_answer = np.zeros(exp_answer.shape) # (anchor, reference, MIX, PM)
for i in range(len(sorted_answer)):
    o1 = exp_order[3*i]
    o2 = exp_order[3*i+1]
    o3 = exp_order[3*i+2]
    a = exp_answer[i]

    cnt = 0
    for j in o1:
        sorted_answer[i, j] = a[cnt]
        cnt += 1
    for j in o2:
        sorted_answer[i, j+4] = a[cnt]
        cnt += 1
    for j in o3:
        sorted_answer[i, j+8] = a[cnt]
        cnt += 1

q1 = sorted_answer[:,0:4]
q2 = sorted_answer[:,4:8]
q3 = sorted_answer[:,8:12]
pra = pra[:,[2,3,0,1]]
q1,q2,q3

In [ ]:
# Extract data
# q1: Other(Instrumental) q2: Drums q3: Vocals
qn = q1
qn = np.delete(qn,1,axis=0)

MIX_score = []
PM_score = []
anch_score = []
ref_score = []
for a in qn:
#     if a[0] >= 1000:
#         pass
#     elif a[1] < 0:
#         pass
#     elif a[3] == 100 or a[3] == 50:
#         pass
#     elif a[2] == 60:
#         pass
#     else:
    MIX_score.append(a[2])
    PM_score.append(a[3])
    anch_score.append(a[0])
    ref_score.append(a[1])  

MIX_score = np.array(MIX_score)
PM_score = np.array(PM_score)
anch_score = np.array(anch_score)
ref_score = np.array(ref_score)
print(MIX_score)
print(PM_score)

In [ ]:
data = [anch_score, PM_score, MIX_score, ref_score]

# Boxplot
fig, ax = plt.subplots(figsize=(4.5,4))
# ax.set_title("")
info = ax.boxplot(data, notch=True)
ax.set_xticklabels(["C1","C2", "C3", "C4"])
ax.set_xlabel("Label")
ax.set_ylabel("Score")
plt.tight_layout()
plt.savefig("figs/boxplot_vocals.pdf")
plt.show()

# Histgram
fig, ax = plt.subplots(1,len(data),figsize=(12,3))
for n in range(len(data)):
    ax[n].hist(data[n],bins=20,range=(0,100))
    ax[n].set_xlim(0,100)
plt.show()


In [ ]:
# Delete outliers
MIX_score_ = MIX_score[MIX_score!=60]
print(len(MIX_score_),len(MIX_score))
PM_score_ = PM_score[PM_score!=50]
PM_score_ = PM_score_[PM_score_!=0]
print(len(PM_score_),len(PM_score))

# Shapiro-Wilk test
print(stats.shapiro(MIX_score_))
print(stats.shapiro(PM_score_))

# Welch's t test
stats.ttest_ind(MIX_score_, PM_score_, equal_var=False)